# Gravit Trust Propagation Simulation

Interactive Colab notebook for testing Gravit trust propagation scenarios.
Generates charts, cascade visualizations, and comprehensive reports.

## 1. Install Required Libraries

In [ ]:
!pip install matplotlib ipywidgets --quiet

## 2. Import Libraries

In [ ]:
import json, random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from ipywidgets import interact, IntSlider
from datetime import datetime

## 3. Create Test Scenarios

In [ ]:
scenarios = [
    {"id": "S1", "type": "Network Trust", "description": "Initial network trust propagation scenario."},
    {"id": "S2", "type": "Consensus Test", "description": "Testing Gravit Consensus trust dynamics."},
    {"id": "S3", "type": "Quantum Layer", "description": "Simulating Gravit Quantum Platform trust propagation."}
]

with open("scenarios.json","w") as f:
    json.dump(scenarios,f,indent=2)

## 4. Trust Simulation Function

In [ ]:
def run_simulation(scenario):
    base=random.uniform(0.55,0.95)
    delta=random.uniform(-0.25,0.1)
    return {
        "avg_before": base,
        "avg_after": max(0.0,min(1.0,base+delta)),
        "min_before": max(0.0,base-random.uniform(0.05,0.2)),
        "max_before": min(1.0,base+random.uniform(0.05,0.2)),
        "min_after": max(0.0,base+delta-random.uniform(0.05,0.2)),
        "max_after": min(1.0,base+delta+random.uniform(0.05,0.2)),
        "notes": f"Simulated scenario {scenario['id']}",
        "type": scenario.get("type","N/A"),
        "description": scenario.get("description","")
    }

## 5. Time-Series Generation

In [ ]:
def generate_time_series(result, steps=20):
    t=np.linspace(0,10,steps)
    y=result["avg_before"]+(result["avg_after"]-result["avg_before"])*(t/t[-1])
    return t,y

## 6. Setup Output Directory

In [ ]:
figures_dir = Path("figures")
figures_dir.mkdir(exist_ok=True)

## 7. Chart Generation Functions

In [ ]:
def generate_trust_chart(sid, result):
    t,y=generate_time_series(result)
    plt.figure(figsize=(7,4))
    plt.plot(t,y,marker='o',label=f"{sid} Avg Trust")
    plt.fill_between(t,result["min_before"],result["max_before"],color='blue',alpha=0.1,label="Min/Max Before")
    plt.fill_between(t,result["min_after"],result["max_after"],color='green',alpha=0.1,label="Min/Max After")
    plt.title(f"Trust Propagation - Scenario {sid}")
    plt.xlabel("Time Steps")
    plt.ylabel("Trust Value")
    plt.ylim(0,1)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(figures_dir/f"{sid}_trust.png")
    plt.close()

def generate_cascade_text(sid):
    depth=random.randint(2,5)
    lines=["Root Node"]+[("  "*i)+f"-> Node {i}" for i in range(1,depth+1)]
    Path(figures_dir/f"{sid}_cascade.txt").write_text("\n".join(lines),encoding="utf-8")

## 8. Run Simulation for All Scenarios

In [ ]:
results={sc["id"]:run_simulation(sc) for sc in scenarios}
scenario_ids=list(results.keys())

for sid,r in results.items():
    generate_trust_chart(sid,r)
    generate_cascade_text(sid)

print(f"Generated {len(results)} simulation results and charts.")

## 9. Interactive Visualization

Use the slider to explore different scenarios interactively.

In [ ]:
def interactive_plot(idx):
    sid=scenario_ids[idx]
    r=results[sid]
    t,y=generate_time_series(r)

    plt.figure(figsize=(7,4))
    plt.plot(t,y,marker='o',label=f"{sid} Avg Trust")
    plt.fill_between(t,r["min_before"],r["max_before"],color='blue',alpha=0.1,label="Min/Max Before")
    plt.fill_between(t,r["min_after"],r["max_after"],color='green',alpha=0.1,label="Min/Max After")
    plt.title(f"Interactive Trust Simulation - Scenario {sid}")
    plt.xlabel("Time Steps")
    plt.ylabel("Trust Value")
    plt.ylim(0,1)
    plt.grid(True)
    plt.legend()
    plt.show()

    depth=random.randint(2,5)
    print("Cascade Propagation (Text):")
    print("Root Node")
    for i in range(1,depth+1):
        print("  "*i+f"-> Node {i}")

    print(f"\nNotes: {r['notes']}\nType: {r['type']}\nDescription: {r['description']}")

interact(interactive_plot, idx=IntSlider(min=0,max=len(scenario_ids)-1,step=1,description="Scenario Index"))

## 10. Export Results (JSON & Markdown Report)

In [ ]:
# Export JSON
Path("aggregated_results.json").write_text(json.dumps(results,indent=2),encoding="utf-8")

# Generate Markdown Report
lines=[]
append=lines.append
append("# Gravit Simulation — Aggregated Report")
append(f"Date: {datetime.utcnow().strftime('%Y-%m-%d')}\n")
append("| Scenario ID | Type | Description | Avg Before | Avg After | Notes | Graph |")
append("|-------------|------|-------------|------------|-----------|-------|-------|")

for sid,r in results.items():
    g=figures_dir/f"{sid}_trust.png"
    gm=f"![{sid}]({g.as_posix()})" if g.exists() else "N/A"
    append(f"| {sid} | {r['type']} | {r['description']} | {r['avg_before']:.4f} | {r['avg_after']:.4f} | {r['notes']} | {gm} |")

for sid,r in results.items():
    append(f"\n### Scenario {sid}\n**Type:** {r['type']}\n**Description:** {r['description']}\n**Notes:** {r['notes']}")
    cascade_txt=figures_dir/f"{sid}_cascade.txt"
    if cascade_txt.exists():
        append("```\n"+cascade_txt.read_text()+"\n```")
    g=figures_dir/f"{sid}_trust.png"
    if g.exists():
        append(f"![Trust Chart]({g.as_posix()})")

Path("aggregated_report_final.md").write_text("\n".join(lines),encoding="utf-8")

## 11. Automatic Google Drive Storing

In [ ]:
notebook_name = "Gravit_Simulation_Colab_with_delay.ipynb"
save_path = f"/content/drive/MyDrive/{notebook_name}"
shutil.copy(notebook_name, save_path)
print(f"✅ Notebook saved to your Google Drive: {save_path}")

## 12. Final pause

In [ ]:
print("✅ Simulation complete. Waiting for 5 seconds before finishing...")
time.sleep(5)
print("Done.")